# Tutorial

In [ ]:
# import all packages you will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.signal
import datetime
import glob
import math
import sys

In [ ]:
import sys, os
# Get the current script’s directory
current_directory = os.getcwd()
# Go back one folder level
parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
sys.path.insert(0, parent_directory)
from mtsthelens import preprocessing_functions, manipulation_functions, plotting_functions

## Read & preprocess data

In [ ]:
# df =  preprocessing_functions.read_data('../data/DSAR_extended2_long.csv')
df =  preprocessing_functions.read_data('../example/example_data/example_data_eruption.csv')
df

In [ ]:
# data smoothing
df = df.rolling('6H', center=True).median()

In [ ]:
# remove outliers
df = df.apply(preprocessing_functions.mask_df,axis=0) # remove outliers (peak detection)

## Data manipulation

### Stack in Time

In [ ]:
df_stackTime = manipulation_functions.stackInTime(df)
manipulation_functions.export_csv('example_data_eruption_stacktime',df_stackTime)
df_stackTime

### Stack in Space

In [ ]:
df_median_stackSpace, df_stackSpace_year = manipulation_functions.stackInSpace(df)
df_yearlyParam = manipulation_functions.stackSpace_yearParam(df_stackSpace_year)
manipulation_functions.export_csv('example_data_eruption_stackspaceYear',df_stackSpace_year)
manipulation_functions.export_csv('example_data_eruption_stackspaceParam',df_yearlyParam)

### Apply Filter

In [ ]:
df = df.fillna(0)
df_filter = manipulation_functions.filter_data(df)
manipulation_functions.export_csv('example_data_eruption_filter',df_filter)
print(df_filter)
plt.figure()
plt.plot(df_filter)

## Data plotting

In [ ]:
# read extrusion rate data
df_dome = pd.read_csv('../example/example_data/dome_extrusion.txt', header=0, skiprows=0)
df_dome.set_index('Date of photography',inplace=True)
df_dome.index = pd.to_datetime(df_dome.index).tz_localize(None)
df_dome['diff'] = df_dome['Total volume change(x 106 m3)']-df_dome['Total volume change(x 106 m3)'].shift(1)
df_dome

In [ ]:
df_dome.plot(marker='o')

In [ ]:
# load station coordinates and drop the stations which are not of interresst
sta_list =['BLIS', 'CDF', 'EDM', 'ELK', 'FL2', 'HOA', 'HSR', 'JRO', 'JUN', # specify the stations you want to use
           'LOO', 'MIDE', 'NED', 'RAFT', 'REM', 'SEP', 'SHW', 'SOS', 'SPN5',
           'STD', 'SUG', 'SWFL', 'TDL', 'USFR', 'VALT', 'YEL'] 

df_sta = pd.read_csv('./example_data/sta_log_long.txt', sep='|', header=0) # coordinates
df = df_sta[~df_sta['Station'].isin(list(set(df_sta.Station)-set(sta_list)))] # delete Stations which are not of interresst
df_sta = df_sta.drop_duplicates(subset=['Station']) # drop one station if the station is not unique
df_sta = df_sta.reset_index(drop=True)
df_sta

In [ ]:
# get the distance between the stations and sort them in increasing order (relative to station SEP -> crater center)
ref_sta = 'SEP' # define the reference station, we will get distance from all other stations to this station
df_sta['dist'] = df_sta.apply(lambda x: preprocessing_functions.calculate_distance(x['latitude'],df_sta.latitude[df_sta['Station']==ref_sta] , x['longitude'],df_sta.longitude[df_sta['Station']==ref_sta] ), axis=1)
df_sta = df_sta.sort_values(by=['dist'])
sta_sorted = df_sta['Station'].to_list()
df_sta